In [2]:
import os
imdb_dir = 'aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [3]:
len(texts)

25000

In [4]:
len(labels)

25000

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
maxlen = 100
training_samples = 200
validation_samples = 10000
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

In [6]:
word_index = tokenizer.word_index

In [7]:
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

Shape of data tensor: (25000, 100)
Shape of label tensor: (25000,)


In [8]:
data

array([[2445,   20,    1, ..., 4782,   22, 2766],
       [  14,   51,  196, ...,   24, 9366, 4130],
       [ 915,   80,  270, ...,   74,   35,   49],
       ...,
       [   0,    0,    0, ...,    3, 1120,  778],
       [3026, 3316,    8, ...,    1, 1960, 7441],
       [ 118,  237,  620, ...,    5,   76,  142]], dtype=int32)

In [9]:
len(data[0])

100

In [10]:
labels

array([1, 1, 0, ..., 0, 0, 0])

In [11]:
training_samples

200

In [12]:
glove_dir = ''
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [13]:
embedding_matrix = np.zeros((max_words, 100))

In [14]:
embedding_matrix.shape

(10000, 100)

In [15]:
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
embedding_dim = 100
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1000000   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                320032    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,320,065
Trainable params: 1,320,065
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = True

In [19]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])

In [20]:
history = model.fit(x_train, y_train,
epochs=10,
batch_size=32,
validation_data=(x_val, y_val))

Epoch 1/10
7/7 [==============================] - 4s 555ms/step - loss: 2.0024 - acc: 0.4800 - val_loss: 0.6930 - val_acc: 0.5073
Epoch 2/10
7/7 [==============================] - 2s 349ms/step - loss: 0.6568 - acc: 0.6000 - val_loss: 0.7239 - val_acc: 0.5069
Epoch 3/10
7/7 [==============================] - 2s 308ms/step - loss: 0.5972 - acc: 0.6450 - val_loss: 1.0694 - val_acc: 0.5071
Epoch 4/10
7/7 [==============================] - 3s 452ms/step - loss: 0.5609 - acc: 0.6600 - val_loss: 0.6895 - val_acc: 0.5421
Epoch 5/10
7/7 [==============================] - 3s 376ms/step - loss: 0.5296 - acc: 0.7700 - val_loss: 0.7655 - val_acc: 0.5078
Epoch 6/10
7/7 [==============================] - 3s 445ms/step - loss: 0.4830 - acc: 0.7250 - val_loss: 0.9320 - val_acc: 0.5074
Epoch 7/10
7/7 [==============================] - 2s 350ms/step - loss: 0.4454 - acc: 0.7500 - val_loss: 0.6948 - val_acc: 0.5323
Epoch 8/10
7/7 [==============================] - 3s 371ms/step - loss: 0.4251 - acc: 0.83